In [10]:
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
from time import sleep
import datetime
import requests
import csv

In [3]:
main_url = "https://www.kv.ee/?act=search.simple&last_deal_type=&company_id=&page=1&orderby=ob&page_size=50&deal_type=1&dt_select=1&county=1&search_type=new&parish=1061&rooms_min=&rooms_max=&price_min=&price_max=&nr_of_people=&area_min=&area_max=&floor_min=&floor_max=&energy_certs=&keyword="
result = requests.get(main_url)
print(result.content[:100])

b'<!DOCTYPE html>\n<html lang="et" dir="ltr" class="no-js">\n<head>\n    <meta charset="utf-8" />\n    <me'


In [4]:
soup = BeautifulSoup(result.content, "html.parser")

In [5]:
soup.title.text

'Kinnisvara KV.EE -  Kinnisvara pakkumised - korterid majad maad äripinnad.\xa0 - Müüa korterid '

In [6]:
data = []

In [7]:
total_pages = soup.select_one('a.count').text
urls = []

for x in range(1, int(total_pages) + 1):
    urls.append("https://www.kv.ee/?act=search.simple&last_deal_type=&company_id=&page=" + str(x) + "&orderby=ob&page_size=50&deal_type=1&dt_select=1&county=1&search_type=new&parish=1061&rooms_min=&rooms_max=&price_min=&price_max=&nr_of_people=&area_min=&area_max=&floor_min=&floor_max=&energy_certs=&keyword=")

In [8]:
for url in tqdm_notebook(urls):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    scrape_data(soup)
    sleep(randint(1,3))

In [1]:
def scrape_data(soup):

    rows = soup.select('tbody tr.object-item')

    materials = ['kivimaja', 'puitmaja', 'paneelmaja', 'palkmaja']
    conditions = ['uus', 'heas korras', 'renoveeritud', 'san. remont tehtud', 'keskmises seisukorras', 'vajab san. remonti', 'vajab renoveerimist']
    current_year = 2019

    for row in rows:
        d = {}

        address = row.select_one('td.object-name h2 a').text.strip().split(', ')
        description = row.select_one('td.object-name p.object-excerpt').text.strip().split(', ')
        
        d['street'] = address[-1]
        d['district'] = address[2]
        d['rooms'] = row.select_one('td.object-rooms').text
        d['area'] = ''.join(filter(lambda x: x in '.0123456789', row.select_one('td.object-m2').text.strip()))
        d['price'] = ''.join(filter(lambda x: x in '.0123456789', row.select_one('td.object-price p.object-price-value').text.strip()))
        d['floor'] = ''.join(c for c in description[0].split('/')[0] if c.isdigit())
        d['building_floors'] = '-'
        d['type'] = '-'
        d['material'] = '-'
        d['condition'] = '-'
        d['year_built'] = '-'

        if len(description[0].split('/')) > 1:
            d['building_floors'] = description[0].split('/')[1]
            
        for e in description:
            if e in materials:
                d['material'] = e
            if e in conditions:
                d['condition'] = e
            if e.startswith('ehitusaasta'):
                d['year_built'] = ''.join(c for c in e if c.isdigit())
        data.append(d)

In [12]:
file_name = 'kv_tallinn_' + datetime.datetime.now().strftime("%Y-%m-%d") + '.csv'

with open(file_name, 'w') as csv_file:
    writer = csv.DictWriter(csv_file, data[0].keys())
    writer.writeheader()
    for e in data:
        writer.writerow(e)